In [3]:
# Step 1: Import required libraries
import requests
import os
from datetime import datetime
from email.mime.text import MIMEText
import smtplib



In [4]:
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",     
    "ids": "bitcoin",         
    "price_change_percentage": "24h,7d",  
    "per_page": 1,
    "page": 1,
    "sparkline": False
}

r = requests.get(url, params=params)
print(r.status_code)   # return 200
print(r.json())        # see response


200
[{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 108481, 'market_cap': 2159273582070, 'market_cap_rank': 1, 'fully_diluted_valuation': 2159273582070, 'total_volume': 49002197665, 'high_24h': 112533, 'low_24h': 108103, 'price_change_24h': -3731.687007721528, 'price_change_percentage_24h': -3.32554, 'market_cap_change_24h': -75140821515.80176, 'market_cap_change_percentage_24h': -3.36289, 'circulating_supply': 19913359.0, 'total_supply': 19913359.0, 'max_supply': 21000000.0, 'ath': 124128, 'ath_change_percentage': -12.61236, 'ath_date': '2025-08-14T00:37:02.582Z', 'atl': 67.81, 'atl_change_percentage': 159867.79401, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2025-08-29T19:41:51.947Z', 'price_change_percentage_24h_in_currency': -3.3255412180347, 'price_change_percentage_7d_in_currency': -7.221641390730358}]


In [5]:
r = requests.get(url, params=params)

r

<Response [200]>

In [6]:
r = requests.get(url, params=params)
print(type(r.text))
print(r.text)

<class 'str'>
[{"id":"bitcoin","symbol":"btc","name":"Bitcoin","image":"https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400","current_price":108481,"market_cap":2159273582070,"market_cap_rank":1,"fully_diluted_valuation":2159273582070,"total_volume":49002197665,"high_24h":112533,"low_24h":108103,"price_change_24h":-3731.687007721528,"price_change_percentage_24h":-3.32554,"market_cap_change_24h":-75140821515.80176,"market_cap_change_percentage_24h":-3.36289,"circulating_supply":19913359.0,"total_supply":19913359.0,"max_supply":21000000.0,"ath":124128,"ath_change_percentage":-12.61236,"ath_date":"2025-08-14T00:37:02.582Z","atl":67.81,"atl_change_percentage":159867.79401,"atl_date":"2013-07-06T00:00:00.000Z","roi":null,"last_updated":"2025-08-29T19:41:51.947Z","price_change_percentage_24h_in_currency":-3.3255412180347,"price_change_percentage_7d_in_currency":-7.221641390730358}]


In [7]:
print(r.encoding)
print(r.status_code)

utf-8
200


In [8]:
def get_btc_markets():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",     
        "ids": "bitcoin",         
        "price_change_percentage": "24h,7d",  
        "per_page": 1,
        "page": 1,
        "sparkline": False
    }
    headers = {"accept": "application/json", "User-Agent": "Mozilla/5.0"}
    r = requests.get(url, params=params, headers=headers, timeout=20)
    r.raise_for_status()
    data = r.json()
    if not data:
        raise RuntimeError("Empty markets response")
    item = data[0]  # list → first object 
    return {
        "price": item["current_price"],
        "pct_24h": item.get("price_change_percentage_24h_in_currency"),
        "pct_7d": item.get("price_change_percentage_7d_in_currency")
    }


In [9]:
def send_alert(message):
    print(f"[{datetime.now()}] ALERT: {message}")

In [ ]:
!pip install dotenv
from dotenv import

load_dotenv()
EMAIL_SENDER = os.environ.get("EMAIL_SENDER")
EMAIL_RECEIVER = os.environ.get("EMAIL_RECEIVER")
EMAIL_PASSWORD = os.environ.get("EMAIL_PASSWORD")  # Gmail App Password

# =========================
#  Fetch BTC Market Data
# =========================
def get_btc_markets():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "ids": "bitcoin",
        "price_change_percentage": "24h,7d",
        "per_page": 1,
        "page": 1,
        "sparkline": False
    }
    headers = {"accept": "application/json", "User-Agent": "Mozilla/5.0"}
    r = requests.get(url, params=params, headers=headers, timeout=20)
    r.raise_for_status()
    data = r.json()
    if not data:
        raise RuntimeError("Empty markets response")
    item = data[0]
    return {
        "price": item["current_price"],
        "pct_24h": item.get("price_change_percentage_24h_in_currency"),
        "pct_7d": item.get("price_change_percentage_7d_in_currency")
    }

# =========================
#  Send Email Alert
# =========================
def send_email_alert(subject, message):
    #EMAIL_SENDER = os.environ.get("EMAIL_SENDER")
    #EMAIL_RECEIVER = os.environ.get("EMAIL_RECEIVER")
    #EMAIL_PASSWORD = os.environ.get("EMAIL_PASSWORD")  # Gmail App Password
    #if not (EMAIL_SENDER and EMAIL_RECEIVER and EMAIL_PASSWORD):
        #print("❌ Missing environment variables for email config")
       # return
    
    try:
        msg = MIMEText(message)
        msg["Subject"] = subject
        msg["From"] = EMAIL_SENDER
        msg["To"] = EMAIL_RECEIVER

        with smtplib.SMTP_SSL("smtp.gmail.com", 465) as server:
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.sendmail(EMAIL_SENDER, EMAIL_RECEIVER, msg.as_string())
        
        print(f"[{datetime.now()}] ✅ Email alert sent")
    except Exception as e:
        print(f"[{datetime.now()}] ❌ Email failed: {e}")

# =========================
#  Check thresholds + trigger alerts
# =========================
def check_alerts():
    btc = get_btc_markets()
    price = btc["price"]
    pct_24h = btc["pct_24h"]
    pct_7d = btc["pct_7d"]

    print(f"[{datetime.now()}] BTC ${price} | 24h: {pct_24h:.2f}% | 7d: {pct_7d:.2f}%")

    alerts = []

    # Example rules — adjust thresholds as needed
    if pct_24h is not None and pct_24h <= -10:
        alerts.append(f"⚠️ BTC down {pct_24h:.2f}% in last 24h! Current price: ${price}")
    if pct_7d is not None and pct_7d <= -20:
        alerts.append(f"🚨 BTC down {pct_7d:.2f}% in last 7 days! Current price: ${price}")

    if alerts:
        send_email_alert("BTC Market Alert", "\n".join(alerts))

# =========================
#  Main
# =========================

btc = get_btc_markets()
pct_24h = btc["pct_24h"]   
pct_24h = -.48

#type(pct_24h)

if pct_24h > -.49:
    send_email_alert("Success", "It dipped")
    
    print(send_email_alert("Success", "It dipped"))

[2025-08-29 15:42:36.856460] ✅ Email alert sent
[2025-08-29 15:42:37.763281] ✅ Email alert sent
None


In [12]:
EMAIL_SENDER = "teamwaveriders1@gmail.com"
EMAIL_RECEIVER = "belkisrochez@gmail.com"
EMAIL_PASSWORD = "fkbm tiea mhbr uvre"

In [ ]:
import smtplib
import ssl
import os
import requests
import time
from datetime import datetime
from email.mime.text import MIMEText
from dotenv import load_dotenv

# --- Configuration & Pre-flight Checks ---

# Load environment variables from a .env file.
# This file MUST be in the same directory as your script.
load_dotenv()

# --- Environment Variables ---
SENDER_EMAIL = os.environ.get("EMAIL_SENDER")
RECEIVER_EMAIL = os.environ.get("EMAIL_RECEIVER")
EMAIL_PASSWORD = os.environ.get("EMAIL_PASSWORD") # IMPORTANT: Must be a Gmail App Password

# --- Constants ---
BASELINE_FILE = "baseline_price.txt"
DIP_THRESHOLD = -0.001  # Alert when price drops by 5%

# =========================
#  Initial Sanity Checks
# =========================
def check_environment_variables():
    """Checks if all required environment variables are loaded."""
    if not SENDER_EMAIL or not RECEIVER_EMAIL or not EMAIL_PASSWORD:
        print("="*50)
        print("❌ FATAL ERROR: Missing one or more environment variables.")
        print("Please check your .env file and ensure it contains:")
        print("EMAIL_SENDER=\"your_email@gmail.com\"")
        print("EMAIL_RECEIVER=\"recipient_email@example.com\"")
        print("EMAIL_PASSWORD=\"your_16_digit_app_password\"")
        print("="*50)
        return False
    print("✅ Environment variables loaded successfully.")
    return True

# =========================
#  Fetch BTC Market Data
# =========================
def get_btc_markets():
    """Fetches the current market data for Bitcoin from CoinGecko."""
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {"vs_currency": "usd", "ids": "bitcoin", "price_change_percentage": "24h,7d"}
    headers = {"accept": "application/json", "User-Agent": "Mozilla/5.0"}
    try:
        r = requests.get(url, params=params, headers=headers, timeout=20)
        r.raise_for_status()
        data = r.json()
        if not data:
            raise RuntimeError("Empty markets response from API")
        item = data[0]
        return {
            "price": item["current_price"],
            "pct_24h": item.get("price_change_percentage_24h_in_currency"),
            "pct_7d": item.get("price_change_percentage_7d_in_currency")
        }
    except requests.exceptions.RequestException as e:
        print(f"[{datetime.now()}] ❌ API request failed: {e}")
        return None

# =========================
#  Send Email Alert
# =========================
def send_email_alert(subject, message):
    """Sends an email using Gmail's SMTP server."""
    print(f"[{datetime.now()}] 📧 Attempting to send email to {RECEIVER_EMAIL}...")
    
    # This check is redundant if check_environment_variables() passed, but it's good practice.
    if not all([SENDER_EMAIL, RECEIVER_EMAIL, EMAIL_PASSWORD]):
        print(f"[{datetime.now()}] ❌ Cannot send email due to missing credentials.")
        return

    try:
        msg = MIMEText(message, 'plain')
        msg["Subject"] = subject
        msg["From"] = SENDER_EMAIL
        msg["To"] = RECEIVER_EMAIL

        context = ssl.create_default_context()
        with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
            print(f"[{datetime.now()}] Logging into SMTP server...")
            server.login(SENDER_EMAIL, EMAIL_PASSWORD)
            print(f"[{datetime.now()}] Sending email...")
            server.sendmail(SENDER_EMAIL, RECEIVER_EMAIL, msg.as_string())
        
        print(f"[{datetime.now()}] ✅ Email alert sent successfully!")
    except smtplib.SMTPAuthenticationError:
        print(f"[{datetime.now()}] ❌ SMTP Authentication Error! Failed to send email.")
        print("    - Check if your EMAIL_PASSWORD is a valid 16-digit Gmail App Password.")
        print("    - Ensure your SENDER_EMAIL is correct.")
    except Exception as e:
        print(f"[{datetime.now()}] ❌ An unexpected error occurred while sending email: {e}")

# ... (The rest of your functions: check_api_alerts, check_dip_alert remain the same) ...

def check_api_alerts(btc_data):
    price = btc_data["price"]
    pct_24h = btc_data["pct_24h"]
    pct_7d = btc_data["pct_7d"]
    print(f"[{datetime.now()}] BTC ${price:,.2f} | 24h: {pct_24h or 0:.2f}% | 7d: {pct_7d or 0:.2f}%")
    alerts = []
    if pct_24h is not None and pct_24h <= -10:
        alerts.append(f"⚠️ BTC down {pct_24h:.2f}% in last 24h! Current price: ${price:,.2f}")
    if pct_7d is not None and pct_7d <= -20:
        alerts.append(f"🚨 BTC down {pct_7d:.2f}% in last 7 days! Current price: ${price:,.2f}")
    if alerts:
        send_email_alert("BTC Market Alert", "\n".join(alerts))

def check_dip_alert(current_price):
    baseline_price = 0.0
    try:
        with open(BASELINE_FILE, "r") as f:
            baseline_price = float(f.read())
    except (FileNotFoundError, ValueError):
        print(f"[{datetime.now()}] No baseline found. Setting baseline price to ${current_price:,.2f}")
        with open(BASELINE_FILE, "w") as f:
            f.write(str(current_price))
        return
    if baseline_price > 0:
        percentage_change = ((current_price - baseline_price) / baseline_price) * 100
    else:
        percentage_change = 0
    print(f"[{datetime.now()}] Current Price: ${current_price:,.2f}, Baseline: ${baseline_price:,.2f}, Change: {percentage_change:.2f}%")
    if percentage_change <= DIP_THRESHOLD:
        print(f"[{datetime.now()}] ❗️ Dip detected! Price dropped by {percentage_change:.2f}%. Sending alert...")
        subject = f"⚠️ Bitcoin Price Alert — BTC is down {abs(percentage_change):.2f}%!"
        body = f"""
Hello,

Bitcoin has dropped by {percentage_change:.2f}% from its baseline price.

📉 Current Price: ${current_price:,.2f}
📊 Baseline Price: ${baseline_price:,.2f}

💡 Suggestion: Adjust your Dollar Cost Average (DCA) plan if desired.

Stay disciplined,
DipDetector by WaveRiders
"""
        send_email_alert(subject, body)
        print(f"[{datetime.now()}] Updating baseline price to ${current_price:,.2f}")
        with open(BASELINE_FILE, "w") as f:
            f.write(str(current_price))

# =========================
#  Main Loop
# =========================
def main():
    """Main function to run the alert checker in a loop."""
    print("🚀 Starting Bitcoin Price Alerter...")
    if not check_environment_variables():
        return # Stop execution if environment is not set up correctly

    while True:
        try:
            btc_data = get_btc_markets()
            if btc_data:
                check_api_alerts(btc_data)
                check_dip_alert(btc_data["price"])
        except Exception as e:
            print(f"[{datetime.now()}] An unexpected error occurred in the main loop: {e}")
        
        print("\n--- Waiting for 1 minute... ---\n")
        time.sleep(60)

if __name__ == "__main__":
    main()
    
    send_email_alert(""""

Bitcoin has dropped by {percentage_change:.2f}% from its baseline price.

📉 Current Price: ${current_price:,.2f}
📊 Baseline Price: ${baseline_price:,.2f}

💡 Suggestion: Adjust your Dollar Cost Average (DCA) plan if desired.

Stay disciplined,
DipDetector by WaveRiders
""")
       
    print(send_email_alert(""""

Bitcoin has dropped by {percentage_change:.2f}% from its baseline price.

📉 Current Price: ${current_price:,.2f}
📊 Baseline Price: ${baseline_price:,.2f}

💡 Suggestion: Adjust your Dollar Cost Average (DCA) plan if desired.

Stay disciplined,
DipDetector by WaveRiders
"""))

🚀 Starting Bitcoin Price Alerter...
✅ Environment variables loaded successfully.
[2025-08-29 16:04:07.328429] BTC $108,155.00 | 24h: -3.37% | 7d: -7.36%
[2025-08-29 16:04:07.333427] Current Price: $108,155.00, Baseline: $108,139.00, Change: 0.01%

--- Waiting for 1 minute... ---

[2025-08-29 16:05:07.453032] BTC $108,155.00 | 24h: -3.37% | 7d: -7.37%
[2025-08-29 16:05:07.455018] Current Price: $108,155.00, Baseline: $108,139.00, Change: 0.01%

--- Waiting for 1 minute... ---

[2025-08-29 16:06:07.559888] BTC $108,100.00 | 24h: -3.42% | 7d: -7.41%
[2025-08-29 16:06:07.560884] Current Price: $108,100.00, Baseline: $108,139.00, Change: -0.04%
[2025-08-29 16:06:07.560884] ❗️ Dip detected! Price dropped by -0.04%. Sending alert...
[2025-08-29 16:06:07.560884] 📧 Attempting to send email to belkisrochez@gmail.com...
[2025-08-29 16:06:10.377529] Logging into SMTP server...
[2025-08-29 16:06:10.795750] Sending email...
[2025-08-29 16:06:12.658489] ✅ Email alert sent successfully!
[2025-08-29 16

: 